In [2]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import deque
import os
import hashlib
import json
import pickle

# Configuration for 4x4 grid
GRID_SIZE = 4
EXIT_POS = (1, GRID_SIZE - 1)  # Exit at row 1 (0-indexed), rightmost column

class RushHour4x4:
    def __init__(self):
        self.grid_size = GRID_SIZE
        self.exit_pos = EXIT_POS
    
    def create_empty_grid(self):
        """Create empty 4x4 grid"""
        return [['.' for _ in range(self.grid_size)] for _ in range(self.grid_size)]
    
    def find_piece_positions(self, grid, piece_id):
        """Find all positions occupied by a piece"""
        positions = []
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] == piece_id:
                    positions.append((r, c))
        return positions
    
    def can_place_piece(self, grid, positions):
        """Check if a piece can be placed at given positions"""
        for r, c in positions:
            if not (0 <= r < self.grid_size and 0 <= c < self.grid_size):
                return False
            if grid[r][c] != '.':
                return False
        return True
    
    def place_piece(self, grid, piece_id, positions):
        """Place a piece on the grid"""
        for r, c in positions:
            grid[r][c] = piece_id
    
    def remove_piece(self, grid, piece_id):
        """Remove a piece from the grid"""
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] == piece_id:
                    grid[r][c] = '.'
    
    def get_all_empty_positions(self, grid):
        """Get all empty positions in the grid"""
        empty = []
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] == '.':
                    empty.append((r, c))
        return empty
    
    def generate_2x1_positions(self, start_pos, orientation):
        """Generate positions for a 2x1 blocker"""
        r, c = start_pos
        if orientation == 'horizontal':
            return [(r, c), (r, c + 1)]
        else:  # vertical
            return [(r, c), (r + 1, c)]
    
    def get_difficulty_config(self, puzzle_num, total_puzzles):
        """
        Get difficulty configuration based on puzzle number.
        Creates a good mix of easy, medium, and hard puzzles.
        """
        # Define difficulty levels with (num_1x1, num_2x1, difficulty_name)
        difficulty_configs = [
            # Easy
            (4, 0, "easy"),
            (3, 1, "easy"),
            (5, 0, "easy"),
            # Medium
            (2, 2, "medium"),
            (3, 2, "medium"),
            (1, 2, "medium"),
            (4, 1, "medium"),
            # Hard
            (0, 3, "hard"),
            (1, 3, "hard"),
            (2, 3, "hard"),
            (0, 4, "hard"),
        ]
        
        # Distribute difficulties across puzzles
        # 40% easy, 40% medium, 20% hard
        if puzzle_num <= total_puzzles * 0.4:
            easy_configs = [config for config in difficulty_configs if config[2] == "easy"]
            return random.choice(easy_configs)
        elif puzzle_num <= total_puzzles * 0.8:
            medium_configs = [config for config in difficulty_configs if config[2] == "medium"]
            return random.choice(medium_configs)
        else:
            hard_configs = [config for config in difficulty_configs if config[2] == "hard"]
            return random.choice(hard_configs)
    
    def generate_puzzle(self, num_1x1_blockers=3, num_2x1_blockers=2, max_attempts=1000):
        """Generate a random 4x4 Rush Hour puzzle"""
        for _ in range(max_attempts):
            grid = self.create_empty_grid()
            
            # Place car (1x1) - not at exit position
            available_positions = [(r, c) for r in range(self.grid_size) 
                                   for c in range(self.grid_size) if (r, c) != self.exit_pos]
            car_pos = random.choice(available_positions)
            self.place_piece(grid, 'C', [car_pos])
            
            # Place 2x1 blockers first
            blockers_placed = 0
            for i in range(num_2x1_blockers):
                piece_id = f'H{i+1}'  # H for 2x1 pieces
                placed = False
                attempts_for_this_piece = 0
                
                while not placed and attempts_for_this_piece < 50:
                    orientation = random.choice(['horizontal', 'vertical'])
                    if orientation == 'horizontal':
                        valid_starts = [(r, c) for r in range(self.grid_size) 
                                        for c in range(self.grid_size - 1)]
                    else:  # vertical
                        valid_starts = [(r, c) for r in range(self.grid_size - 1) 
                                        for c in range(self.grid_size)]
                    
                    if valid_starts:
                        start_pos = random.choice(valid_starts)
                        positions = self.generate_2x1_positions(start_pos, orientation)
                        if self.can_place_piece(grid, positions):
                            self.place_piece(grid, piece_id, positions)
                            placed = True
                            blockers_placed += 1
                    attempts_for_this_piece += 1
            
            # Place 1x1 blockers
            for i in range(num_1x1_blockers):
                piece_id = f'B{i+1}'
                empty_positions = self.get_all_empty_positions(grid)
                if empty_positions:
                    pos = random.choice(empty_positions)
                    self.place_piece(grid, piece_id, [pos])
                    blockers_placed += 1
            
            # Check if we have enough blockers and the puzzle is solvable
            expected_total = num_2x1_blockers + num_1x1_blockers
            if blockers_placed >= max(1, expected_total - 1):  # Allow some flexibility
                solution = self.bfs_solve(grid)
                if solution is not None and len(solution) > 0:  # At least 1 move
                    return grid
        
        raise Exception(f"Could not generate solvable puzzle after {max_attempts} attempts")
    
    def grid_to_tuple(self, grid):
        """Convert grid to hashable tuple"""
        return tuple(tuple(row) for row in grid)
    
    def tuple_to_grid(self, t):
        """Convert tuple back to grid"""
        return [list(row) for row in t]
    
    def is_solved(self, grid):
        """Check if car 'C' is at exit position"""
        return grid[self.exit_pos[0]][self.exit_pos[1]] == 'C'
    
    def get_piece_info(self, grid, piece_id):
        """Get information about a piece (positions and type)"""
        positions = self.find_piece_positions(grid, piece_id)
        if len(positions) == 1:
            return positions, '1x1'
        elif len(positions) == 2:
            r1, c1 = positions[0]
            r2, c2 = positions[1]
            if r1 == r2:
                return sorted(positions, key=lambda x: x[1]), 'horizontal'
            else:
                return sorted(positions, key=lambda x: x[0]), 'vertical'
        return positions, 'unknown'
    
    def get_neighbors(self, state):
        """Get all valid neighboring states (move any piece by 1 in any direction)"""
        grid = self.tuple_to_grid(state)
        neighbors = []
        
        # Find all unique pieces
        pieces = set()
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] != '.':
                    pieces.add(grid[r][c])
        
        # Try moving each piece in all 4 directions
        for piece_id in pieces:
            positions, _ = self.get_piece_info(grid, piece_id)
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # up, down, left, right
                new_positions = [(r + dr, c + dc) for r, c in positions]
                
                # Bounds check
                valid_move = True
                for nr, nc in new_positions:
                    if not (0 <= nr < self.grid_size and 0 <= nc < self.grid_size):
                        valid_move = False
                        break
                if not valid_move:
                    continue
                
                # Collision check
                temp_grid = [row[:] for row in grid]
                self.remove_piece(temp_grid, piece_id)
                can_move = True
                for nr, nc in new_positions:
                    if temp_grid[nr][nc] != '.':
                        can_move = False
                        break
                if not can_move:
                    continue
                
                # Apply move
                new_grid = [row[:] for row in temp_grid]
                self.place_piece(new_grid, piece_id, new_positions)
                neighbors.append(self.grid_to_tuple(new_grid))
        
        return neighbors
    
    def bfs_solve(self, start_grid):
        """BFS solver that returns ONE optimal solution"""
        start_state = self.grid_to_tuple(start_grid)
        if self.is_solved(start_grid):
            return []  # Already solved
        
        queue = deque([(start_state, [])])
        visited = {start_state}
        
        while queue:
            current_state, path = queue.popleft()
            
            for next_state in self.get_neighbors(current_state):
                if next_state in visited:
                    continue
                    
                visited.add(next_state)
                new_path = path + [next_state]
                
                if self.is_solved(self.tuple_to_grid(next_state)):
                    return new_path
                
                queue.append((next_state, new_path))
        
        return None
    
    def verify_solution(self, start_grid, solution_path):
        """Verify that a solution path is valid"""
        if not solution_path:
            return self.is_solved(start_grid)
        
        current = self.grid_to_tuple(start_grid)
        for i, next_state in enumerate(solution_path):
            valid_neighbors = self.get_neighbors(current)
            if next_state not in valid_neighbors:
                print(f"Invalid move at step {i+1}")
                return False
            current = next_state
        
        final_grid = self.tuple_to_grid(current)
        if not self.is_solved(final_grid):
            print("Final state is not solved")
            return False
        return True
    
    def generate_solution_moves(self, start_grid, solution_path):
        """Generate text description of moves"""
        moves = []
        current_grid = start_grid
        
        for i, next_state in enumerate(solution_path):
            next_grid = self.tuple_to_grid(next_state)
            moved_piece = None
            old_positions = []
            new_positions = []
            
            # Unique pieces in current grid
            current_pieces = set()
            for r in range(self.grid_size):
                for c in range(self.grid_size):
                    if current_grid[r][c] != '.':
                        current_pieces.add(current_grid[r][c])
            
            for piece in current_pieces:
                old_pos = self.find_piece_positions(current_grid, piece)
                new_pos = self.find_piece_positions(next_grid, piece)
                if old_pos != new_pos:
                    moved_piece = piece
                    old_positions = old_pos
                    new_positions = new_pos
                    break
            
            if moved_piece:
                # Convert to 1-indexed for display
                old_display = [(r+1, c+1) for r, c in old_positions]
                new_display = [(r+1, c+1) for r, c in new_positions]
                
                if len(old_positions) == 1:
                    moves.append(f"Step {i+1}: {moved_piece} [{old_display[0][0]},{old_display[0][1]}] -> [{new_display[0][0]},{new_display[0][1]}]")
                else:
                    moves.append(f"Step {i+1}: {moved_piece} {old_display} -> {new_display}")
            
            current_grid = next_grid
        
        return moves
    
    def generate_puzzle_json(self, grid, exit_pos=None, difficulty="unknown", num_1x1_blockers=0, num_2x1_blockers=0,
                           solution_length=0, transformation="original"):
        """Generate JSON representation of puzzle state with embedded prompt"""
        if exit_pos is None:
            exit_pos = self.exit_pos
        
        # Find car position
        car_pos = self.find_piece_positions(grid, 'C')[0]
        
        # Convert positions to 1-indexed for JSON (matches visual display)
        car_pos_1indexed = [car_pos[0] + 1, car_pos[1] + 1]
        exit_pos_1indexed = [exit_pos[0] + 1, exit_pos[1] + 1]
        
        # Build pieces dictionary
        pieces = {}
        pieces['C'] = {"type": "car", "position": car_pos_1indexed}
        
        # Find all blockers
        processed_pieces = {'C'}
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                if cell != '.' and cell not in processed_pieces:
                    positions = self.find_piece_positions(grid, cell)
                    if len(positions) == 1:
                        pos_1indexed = [positions[0][0] + 1, positions[0][1] + 1]
                        pieces[cell] = {"type": "1x1_blocker", "position": pos_1indexed}
                    else:
                        pos_1indexed = [[pos[0] + 1, pos[1] + 1] for pos in positions]
                        if positions[0][0] == positions[1][0]:
                            piece_type = "2x1_horizontal_blocker"
                        else:
                            piece_type = "2x1_vertical_blocker"
                        pieces[cell] = {"type": piece_type, "positions": pos_1indexed}
                    processed_pieces.add(cell)
        
        # Generate the prompt for this specific puzzle
        prompt = self.generate_puzzle_specific_prompt(grid)
        
        # Create JSON structure
        puzzle_json = {
            "grid": grid,
            "car_position": car_pos_1indexed,
            "exit_position": exit_pos_1indexed,
            "pieces": pieces,
            "puzzle_info": {
                "difficulty": difficulty,
                "num_1x1_blockers": num_1x1_blockers,
                "num_2x1_blockers": num_2x1_blockers,
                "total_moves_in_solution": solution_length,
                "transformation": transformation,
                "grid_size": f"{self.grid_size}x{self.grid_size}",
                "coordinate_system": "1-indexed, [row,col] format where [1,1] is top-left"
            },
            "prompt": prompt
        }
        
        return puzzle_json
    
    def generate_puzzle_specific_prompt(self, grid):
        """Generate a customized prompt for the specific 4x4 puzzle"""
        car_pos = self.find_piece_positions(grid, 'C')[0]
        car_pos_1indexed = (car_pos[0] + 1, car_pos[1] + 1)
        exit_pos_1indexed = (self.exit_pos[0] + 1, self.exit_pos[1] + 1)
        
        blockers_1x1 = []
        blockers_2x1 = []
        
        pieces = set()
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                if cell != '.' and cell != 'C' and cell not in pieces:
                    pieces.add(cell)
                    positions = self.find_piece_positions(grid, cell)
                    if len(positions) == 1:
                        pos_1indexed = (positions[0][0] + 1, positions[0][1] + 1)
                        blockers_1x1.append(f"{cell} at [{pos_1indexed[0]},{pos_1indexed[1]}]")
                    else:
                        pos_1indexed = [(pos[0] + 1, pos[1] + 1) for pos in positions]
                        positions_str = ", ".join([f"[{p[0]},{p[1]}]" for p in pos_1indexed])
                        if positions[0][0] == positions[1][0]:
                            orientation = "horizontal"
                        else:
                            orientation = "vertical"
                        blockers_2x1.append(f"{cell} ({orientation}) at {positions_str}")
        
        prompt = f"""Task Description: You are given a 4x4 Rush Hour puzzle. Your goal is to move the blue car (labeled "C") from its current position [{car_pos_1indexed[0]},{car_pos_1indexed[1]}] to the TARGET cell at position [{exit_pos_1indexed[0]},{exit_pos_1indexed[1]}] by moving pieces one square at a time.

Puzzle Pieces:
- Car "C" (blue): Currently at position [{car_pos_1indexed[0]},{car_pos_1indexed[1]}] - This is the piece you need to get to the TARGET

- 1x1 Blockers (B1, B2, etc.): Single-cell obstacles that can be moved to clear a path
  {chr(10).join(f"  - {blocker}" for blocker in blockers_1x1) if blockers_1x1 else "  - None present"}

- 2x1 Blockers (H1, H2, etc.): Two-cell obstacles that move as a single unit
  {chr(10).join(f"  - {blocker}" for blocker in blockers_2x1) if blockers_2x1 else "  - None present"}

- TARGET cell: Position [{exit_pos_1indexed[0]},{exit_pos_1indexed[1]}] (marked with red border and "TARGET" label)

Movement Rules:
- Any piece (car "C", 1x1 blockers "B1, B2, etc.", or 2x1 blockers "H1, H2, etc.") can move UP, DOWN, LEFT, or RIGHT
- Each move is exactly ONE square in any direction for the entire piece
- For 2x1 blockers: The entire piece moves together as a unit (both cells move simultaneously)
- Pieces strictly CANNOT move outside the 4x4 grid
- Pieces strictly CANNOT move into occupied squares (i.e. squares that already have another piece)
- At ANY instant, there CANNOT be two pieces occupying the same square
- The same piece can move multiple times in a row if needed
- You win when car "C" reaches the TARGET cell

Strategy:
- The blockers may or may not be in your way - you may need to move them to create a clear path for the car
- 2x1 blockers take up two cells and move as a single unit, so plan their movements carefully
- Sometimes you need to move blockers temporarily to make space, then move them again
- Plan your moves carefully to avoid getting pieces stuck

Coordinate System:
- Use [row,col] format where [1,1] is top-left, [4,4] is bottom-right
- Each cell shows its coordinates in black text: (row,col)
- For 2x1 blockers, both occupied cells are shown in the piece description

Expected Output Format:
Wrap your solution in <solution> tags and provide it as a numbered list of moves in this exact format:

<solution>
Step 1: [PIECE] [start_position] -> [end_position]
Step 2: [PIECE] [start_position] -> [end_position]
...
</solution>

For 1x1 pieces (car "C" and blockers "B1", "B2", etc.):
- Use single coordinate: C [2,1] -> [2,2]

For 2x1 pieces (blockers "H1", "H2", etc.):
- List both coordinates: H1 [[1,1],[1,2]] -> [[2,1],[2,2]]

Example response format:
<solution>
Step 1: C [2,1] -> [2,2]
Step 2: H1 [[1,3],[1,4]] -> [[2,3],[2,4]]  
Step 3: B1 [3,2] -> [3,1]
Step 4: C [2,2] -> [2,4]
</solution>"""
        return prompt
    
    def draw_grid(self, grid, save_path=None, figsize=(8, 8)):
        """Draw the grid with pieces - no title, cleaner appearance"""
        fig, ax = plt.subplots(figsize=figsize)
        ax.set_xlim(0, self.grid_size)
        ax.set_ylim(0, self.grid_size)
        ax.set_xticks(range(self.grid_size + 1))
        ax.set_yticks(range(self.grid_size + 1))
        
        # Draw main grid lines
        for x in range(self.grid_size + 1):
            ax.plot([x, x], [0, self.grid_size], color='black', linewidth=2)
        for y in range(self.grid_size + 1):
            ax.plot([0, self.grid_size], [y, y], color='black', linewidth=2)
        
        ax.invert_yaxis()
        
        # Draw exit target
        er, ec = self.exit_pos
        target_rect = patches.Rectangle((ec, er), 1, 1,
                                        facecolor='none',
                                        edgecolor='red', linewidth=6)
        ax.add_patch(target_rect)
        
        # Color mapping
        colors = {
            'C': 'skyblue',
            'B1': 'lightcoral', 'B2': 'lightgreen', 'B3': 'plum', 
            'B4': 'khaki', 'B5': 'lightsalmon', 'B6': 'lightsteelblue',
            'H1': 'orange', 'H2': 'cyan', 'H3': 'yellow',
            'H4': 'pink', 'H5': 'lightgray', 'H6': 'lavender'
        }
        
        # Collect info
        piece_info = {}
        dotted_lines = []
        drawn_pieces = set()
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                if cell != '.' and cell not in drawn_pieces:
                    positions = self.find_piece_positions(grid, cell)
                    piece_info[cell] = positions
                    drawn_pieces.add(cell)
        
        # Draw pieces
        for piece_id, positions in piece_info.items():
            if len(positions) == 1:
                r, c = positions[0]
                ax.add_patch(
                    patches.Rectangle((c, r), 1, 1,
                                      facecolor=colors.get(piece_id, 'white'),
                                      edgecolor='black', linewidth=2)
                )
                ax.text(c + 0.5, r + 0.25, piece_id, ha='center', va='center',
                        fontsize=16, fontweight='bold')
            else:
                min_r = min(pos[0] for pos in positions)
                max_r = max(pos[0] for pos in positions)
                min_c = min(pos[1] for pos in positions)
                max_c = max(pos[1] for pos in positions)
                
                width = max_c - min_c + 1
                height = max_r - min_r + 1
                
                ax.add_patch(
                    patches.Rectangle((min_c, min_r), width, height,
                                      facecolor=colors.get(piece_id, 'white'),
                                      edgecolor='black', linewidth=2,
                                      zorder=3)
                )
                
                if width == 2:  # Horizontal 2x1
                    line_x = min_c + 1
                    line_y_start = min_r
                    line_y_end = min_r + 1
                    dotted_lines.append([(line_x, line_y_start), (line_x, line_y_end)])
                    label_x = line_x
                    label_y = min_r + 0.5
                else:  # Vertical 2x1
                    line_x_start = min_c
                    line_x_end = min_c + 1
                    line_y = min_r + 1
                    dotted_lines.append([(line_x_start, line_y), (line_x_end, line_y)])
                    label_x = min_c + 0.5
                    label_y = line_y
                
                ax.text(label_x, label_y, piece_id, ha='center', va='center',
                        fontsize=14, fontweight='bold', zorder=5, color='black')
        
        # Dotted lines
        for line in dotted_lines:
            start, end = line
            ax.plot([start[0], end[0]], [start[1], end[1]], 
                    color='black', linewidth=1, linestyle='--', alpha=0.7, zorder=4)
        
        # Coordinates in each cell
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                coord_text = f"({r+1},{c+1})"
                ax.text(c + 0.5, r + 0.8, coord_text, ha='center', va='center',
                        fontsize=12, color='black', fontweight='normal')
        
        # Target label
        ax.text(ec + 0.85, er + 0.5, 'TARGET', ha='right', va='center',
                fontsize=13, fontweight='bold', color='red',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', 
                          alpha=0.95, edgecolor='red', linewidth=2))
        
        ax.axis('off')
        
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            plt.close()
        else:
            plt.tight_layout()
            plt.show()
    
    def get_grid_hash(self, grid):
        """Generate a hash for a grid to check uniqueness - includes exit position for extra safety"""
        grid_str = str(self.grid_to_tuple(grid))
        exit_str = str(self.exit_pos)
        combined_str = grid_str + exit_str
        return hashlib.sha256(combined_str.encode()).hexdigest()  # SHA256 for better collision resistance
    
    # Transformations for augmentations
    def rotate_90_clockwise(self, grid):
        """Rotate grid 90 degrees clockwise"""
        return [[grid[self.grid_size-1-j][i] for j in range(self.grid_size)] for i in range(self.grid_size)]
    
    def flip_horizontal(self, grid):
        """Flip grid horizontally"""
        return [row[::-1] for row in grid]
    
    def flip_vertical(self, grid):
        """Flip grid vertically"""
        return grid[::-1]
    
    def apply_transformation(self, grid, transformation):
        """Apply transformation and return new grid with updated exit position (1-indexed for info)"""
        if transformation == "original":
            return grid, (2, 4)  # 1-indexed
        elif transformation == "90_rotation":
            new_grid = self.rotate_90_clockwise(grid)
            return new_grid, (4, 3)
        elif transformation == "180_rotation":
            new_grid = self.rotate_90_clockwise(self.rotate_90_clockwise(grid))
            return new_grid, (3, 1)
        elif transformation == "270_rotation":
            new_grid = self.rotate_90_clockwise(self.rotate_90_clockwise(self.rotate_90_clockwise(grid)))
            return new_grid, (1, 2)
        elif transformation == "horizontal_flip":
            new_grid = self.flip_horizontal(grid)
            return new_grid, (2, 1)
        elif transformation == "vertical_flip":
            new_grid = self.flip_vertical(grid)
            return new_grid, (3, 4)
        else:
            return grid, (2, 4)
    
    def _internal_exit_for_transform(self, transform_name):
        """Return 0-indexed internal exit position for a transform name."""
        if transform_name == "original":
            return (1, 3)
        elif transform_name == "90_rotation":
            return (3, 2)
        elif transform_name == "180_rotation":
            return (2, 0)
        elif transform_name == "270_rotation":
            return (0, 1)
        elif transform_name == "horizontal_flip":
            return (1, 0)
        elif transform_name == "vertical_flip":
            return (2, 3)
        else:
            return (1, 3)
    
    def generate_master_puzzle_configs(self, num_puzzles=150, random_seed=42):
        """
        CELL 1: Generate master list of ALL unique puzzle configurations.
        This ensures reproducibility and uniqueness across all puzzles.
        """
        print("🎯 CELL 1: Generating Master Puzzle Configuration List")
        print("=" * 60)
        
        # Set seed for reproducibility
        random.seed(random_seed)
        
        transformations = ["original", "90_rotation", "180_rotation", "270_rotation", "horizontal_flip", "vertical_flip"]
        target_variants = num_puzzles * len(transformations)
        
        master_configs = []
        seen_hashes = set()
        
        # Track difficulty distribution
        difficulty_counts = {"easy": 0, "medium": 0, "hard": 0}
        
        base_index = 0
        attempts_total = 0
        attempts_since_progress = 0
        
        MAX_TOTAL_ATTEMPTS = max(5000, target_variants * 30)
        MAX_NO_PROGRESS = max(800, target_variants // 2)
        
        while len(master_configs) < target_variants and attempts_total < MAX_TOTAL_ATTEMPTS and attempts_since_progress < MAX_NO_PROGRESS:
            attempts_total += 1
            attempts_since_progress += 1
            
            try:
                base_index += 1
                num_1x1, num_2x1, difficulty = self.get_difficulty_config(base_index, num_puzzles)
                
                base_puzzle = self.generate_puzzle(num_1x1_blockers=num_1x1, num_2x1_blockers=num_2x1)
                
                # Process each transformation
                for transform_name in transformations:
                    if len(master_configs) >= target_variants:
                        break
                        
                    transformed_grid, new_exit_pos_1indexed = self.apply_transformation(base_puzzle, transform_name)
                    internal_exit_pos = self._internal_exit_for_transform(transform_name)
                    
                    # Temporarily set exit for solving
                    old_exit_pos = self.exit_pos
                    self.exit_pos = internal_exit_pos
                    
                    try:
                        # Check solvability - only get one solution
                        solution = self.bfs_solve(transformed_grid)
                        if solution is None:
                            continue
                        
                        # Check uniqueness
                        grid_hash = self.get_grid_hash(transformed_grid)
                        if grid_hash in seen_hashes:
                            continue
                        
                        # Accept this configuration
                        seen_hashes.add(grid_hash)
                        attempts_since_progress = 0
                        difficulty_counts[difficulty] += 1
                        
                        # Create configuration record
                        config = {
                            'config_id': len(master_configs) + 1,
                            'base_index': base_index,
                            'transformation': transform_name,
                            'grid': transformed_grid,
                            'exit_pos_internal': internal_exit_pos,
                            'exit_pos_display': new_exit_pos_1indexed,
                            'difficulty': difficulty,
                            'num_1x1_blockers': num_1x1,
                            'num_2x1_blockers': num_2x1,
                            'grid_hash': grid_hash,
                            'solution_length': len(solution),
                            'solution': solution
                        }
                        
                        master_configs.append(config)
                        
                        if len(master_configs) % 50 == 0:
                            print(f"✓ Generated {len(master_configs)}/{target_variants} configurations...")
                            print(f"   Difficulty distribution so far: Easy={difficulty_counts['easy']}, Medium={difficulty_counts['medium']}, Hard={difficulty_counts['hard']}")
                    
                    finally:
                        self.exit_pos = old_exit_pos
            
            except Exception as e:
                continue
        
        print(f"\n🎉 Master configuration generation complete!")
        print(f"Generated {len(master_configs)}/{target_variants} unique configurations")
        print(f"Unique hashes verified: {len(seen_hashes)}")
        print(f"Final difficulty distribution: Easy={difficulty_counts['easy']}, Medium={difficulty_counts['medium']}, Hard={difficulty_counts['hard']}")
        
        # Save master configuration list for reproducibility
        master_file = "data/4x4/master_puzzle_configs.json"
        os.makedirs("data/4x4", exist_ok=True)
        
        # Convert to JSON-serializable format
        json_configs = []
        for config in master_configs:
            json_config = config.copy()
            json_config['solution'] = [list(sol) for sol in config['solution']]  # Convert tuples to lists
            json_configs.append(json_config)
        
        with open(master_file, 'w') as f:
            json.dump(json_configs, f, indent=2)
        
        # Also save as pickle for easier loading
        with open("data/4x4/master_puzzle_configs.pkl", 'wb') as f:
            pickle.dump(master_configs, f)
        
        print(f"Master configuration saved to: {master_file}")
        return master_configs
    
    def create_dataset_from_master_configs(self, master_configs, base_folder="data/4x4"):
        """
        CELL 2: Create dataset from pre-generated master configuration list.
        This ensures reproducibility and processes each unique configuration.
        """
        print("\n🎯 CELL 2: Creating Dataset from Master Configurations")
        print("=" * 60)
        
        if not os.path.exists(base_folder):
            os.makedirs(base_folder)
        
        processed_count = 0
        
        for config in master_configs:
            try:
                # Extract configuration data
                config_id = config['config_id']
                base_index = config['base_index']
                transform_name = config['transformation']
                grid = config['grid']
                internal_exit_pos = config['exit_pos_internal']
                display_exit_pos = config['exit_pos_display']
                difficulty = config['difficulty']
                num_1x1 = config['num_1x1_blockers']
                num_2x1 = config['num_2x1_blockers']
                solution = config['solution']
                
                # Set appropriate exit position
                old_exit_pos = self.exit_pos
                self.exit_pos = internal_exit_pos
                
                try:
                    # Create folder name
                    if transform_name == "original":
                        folder_name = f"puzzle{base_index}"
                    else:
                        folder_name = f"puzzle{base_index}_{transform_name}"
                    
                    puzzle_folder = os.path.join(base_folder, folder_name)
                    if not os.path.exists(puzzle_folder):
                        os.makedirs(puzzle_folder)
                    
                    # Generate and save puzzle-specific prompt
                    puzzle_prompt = self.generate_puzzle_specific_prompt(grid)
                    with open(os.path.join(puzzle_folder, "prompt.txt"), 'w') as f:
                        f.write(puzzle_prompt)
                    
                    # Generate and save JSON representation
                    puzzle_json = self.generate_puzzle_json(
                        grid=grid,
                        exit_pos=internal_exit_pos,
                        difficulty=difficulty,
                        num_1x1_blockers=num_1x1,
                        num_2x1_blockers=num_2x1,
                        solution_length=len(solution),
                        transformation=transform_name
                    )
                    with open(os.path.join(puzzle_folder, "puzzle_state.json"), 'w') as f:
                        json.dump(puzzle_json, f, indent=2)
                    
                    # Save initial state image (no title for cleaner appearance)
                    self.draw_grid(grid, save_path=os.path.join(puzzle_folder, "initial_state.png"))
                    
                    # Save final solved state image
                    if solution:
                        final_state = self.tuple_to_grid(solution[-1])
                        self.draw_grid(final_state, save_path=os.path.join(puzzle_folder, "final_solved_state.png"))
                    
                    # Generate solution text
                    moves = self.generate_solution_moves(grid, solution)
                    
                    solution_file = os.path.join(puzzle_folder, "solution.txt")
                    with open(solution_file, 'w') as f:
                        f.write(f"Puzzle: {folder_name}\n")
                        f.write(f"Configuration ID: {config_id}\n")
                        f.write(f"Total moves: {len(solution)}\n")
                        f.write(f"Exit position: [{display_exit_pos[0]},{display_exit_pos[1]}]\n")
                        f.write(f"Transformation: {transform_name}\n")
                        f.write(f"Difficulty: {difficulty} ({num_1x1} 1x1 blockers, {num_2x1} 2x1 blockers)\n")
                        f.write("Coordinate system: [row,col] where [1,1] is top-left (matches visual grid)\n\n")
                        
                        f.write("SOLUTION:\n")
                        f.write("=" * 40 + "\n")
                        for move in moves:
                            f.write(move + "\n")
                    
                    processed_count += 1
                    
                    if processed_count % 50 == 0:
                        print(f"✓ Processed {processed_count}/{len(master_configs)} configurations...")
                
                finally:
                    self.exit_pos = old_exit_pos
            
            except Exception as e:
                print(f"Error processing configuration {config.get('config_id', 'unknown')}: {e}")
                continue
        
        print(f"\n🎉 Dataset creation complete!")
        print(f"Processed {processed_count}/{len(master_configs)} configurations")
        print(f"Dataset saved in '{base_folder}' folder")
        
        return processed_count
    
    def load_master_configs(self, filepath="data/4x4/master_puzzle_configs.pkl"):
        """Load master configuration list from file"""
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except FileNotFoundError:
            print(f"Master config file not found: {filepath}")
            return None
    
    def create_full_dataset(self, num_puzzles=150, random_seed=42):
        """
        Complete workflow: Generate master configs then create dataset.
        This is the main function to call for full dataset generation.
        """
        print("🚀 Starting Full 4x4 Rush Hour Dataset Generation")
        print("=" * 60)
        print(f"Target: {num_puzzles} base puzzles × 6 transformations = {num_puzzles * 6} total puzzles")
        print(f"Random seed: {random_seed} (for reproducibility)")
        print()
        
        # CELL 1: Generate master configuration list
        master_configs = self.generate_master_puzzle_configs(num_puzzles, random_seed)
        
        # CELL 2: Create dataset from master configurations
        processed_count = self.create_dataset_from_master_configs(master_configs)
        
        print(f"\n✅ COMPLETE: Generated {len(master_configs)} unique puzzle configurations")
        print(f"✅ COMPLETE: Processed {processed_count} puzzle files")
        print("\nDataset features:")
        print("- ✅ Reproducible generation using fixed random seed")
        print(f"- ✅ All {len(master_configs)} configurations guaranteed unique")
        print("- ✅ Single optimal solution per puzzle")
        print("- ✅ No intermediate step images (commented out)")
        print("- ✅ Clean images without titles")
        print("- ✅ Master configuration list saved for reference")
        print("- ✅ JSON state files for text-only models")
        print("- ✅ Difficulty levels: Easy (3-5 blockers), Medium (4-6 blockers), Hard (3-7 blockers)")

    def test_solver(self):
        """Test the BFS solver with known cases"""
        print("Testing 4x4 BFS Solver...")
        
        # Test 1: Generate and solve random puzzle
        random.seed(42)
        test_grid = self.generate_puzzle(num_1x1_blockers=3, num_2x1_blockers=1)
        solution = self.bfs_solve(test_grid)
        assert solution is not None, "Test 1 failed: No solution found"
        assert self.verify_solution(test_grid, solution), "Test 1 verification failed"
        print(f"✓ Test 1 passed: Random puzzle solved in {len(solution)} moves")
        
        # Test 2: Generate another puzzle
        random.seed(123)
        test_grid = self.generate_puzzle(num_1x1_blockers=2, num_2x1_blockers=2)
        solution = self.bfs_solve(test_grid)
        assert solution is not None, "Test 2 failed: No solution found"
        assert self.verify_solution(test_grid, solution), "Test 2 verification failed"
        print(f"✓ Test 2 passed: Puzzle solved in {len(solution)} moves")
        
        print("All 4x4 BFS solver tests passed! ✅\n")

# ----- Main Execution -----

if __name__ == "__main__":
    print("Creating 4x4 Rush Hour Dataset with Master Configuration System...")
    print("This ensures reproducibility and uniqueness across all puzzles.")
    print()
    
    game = RushHour4x4()
    
    # Verify BFS solver first
    game.test_solver()
    
    # Option 1: Generate fresh dataset (recommended for first run)
    game.create_full_dataset(num_puzzles=150, random_seed=42)
    
    # Option 2: Load existing master configs and regenerate dataset
    # master_configs = game.load_master_configs()
    # if master_configs:
    #     game.create_dataset_from_master_configs(master_configs)
    # else:
    #     print("No existing master configs found. Run Option 1 first.")

Creating 4x4 Rush Hour Dataset with Master Configuration System...
This ensures reproducibility and uniqueness across all puzzles.

Testing 4x4 BFS Solver...
✓ Test 1 passed: Random puzzle solved in 1 moves
✓ Test 2 passed: Puzzle solved in 6 moves
All 4x4 BFS solver tests passed! ✅

🚀 Starting Full 4x4 Rush Hour Dataset Generation
Target: 150 base puzzles × 6 transformations = 900 total puzzles
Random seed: 42 (for reproducibility)

🎯 CELL 1: Generating Master Puzzle Configuration List
✓ Generated 50/900 configurations...
   Difficulty distribution so far: Easy=50, Medium=0, Hard=0
✓ Generated 100/900 configurations...
   Difficulty distribution so far: Easy=100, Medium=0, Hard=0
✓ Generated 150/900 configurations...
   Difficulty distribution so far: Easy=150, Medium=0, Hard=0
✓ Generated 200/900 configurations...
   Difficulty distribution so far: Easy=200, Medium=0, Hard=0
✓ Generated 250/900 configurations...
   Difficulty distribution so far: Easy=250, Medium=0, Hard=0
✓ Generate